With an IMU, it might be possible to calculate the relative position. I'll do this in this notebook and calculate the error if with a known start position. 

This notebook has not a conclusion or main message (yet). The idea is more to provide some functionality, which might be useful. 

Thanks to https://www.kaggle.com/hrshtt/kalman-filters-and-imus-starter and https://www.kaggle.com/tosinabase/basic-eda-traces-and-features-visualization

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import matplotlib.pyplot as plt

First, I'll clone the github repro since there are some useful functions for calculating the relative position 

In [ ]:
!git clone https://github.com/location-competition/indoor-location-competition-20.git api

In [ ]:
from api.compute_f import *
from api.io_f import * 
from api.visualize_f import * 

In [ ]:
import json
from pathlib import Path

In [ ]:
# I copied these visualization functions mainly from https://www.kaggle.com/hrshtt/kalman-filters-and-imus-starter and modified them slightly. 

def visualize_trajectory(trajectory, floor_plan_filename, width_meter, height_meter, title=None, mode='lines + markers + text', show=False, trajectory2=None):
    """
    Copied and modified from from https://github.com/location-competition/indoor-location-competition-20/blob/master/visualize_f.py

    """
    fig = go.Figure()

    # add trajectory
    size_list = [6] * trajectory.shape[0]
    size_list[0] = 10
    size_list[-1] = 10

    color_list = ['rgba(4, 174, 4, 0.5)'] * trajectory.shape[0]
    color_list[0] = 'rgba(12, 5, 235, 1)'
    color_list[-1] = 'rgba(235, 5, 5, 1)'
    
    

    position_count = {}
    text_list = []
    for i in range(trajectory.shape[0]):
        if str(trajectory[i]) in position_count:
            position_count[str(trajectory[i])] += 1
        else:
            position_count[str(trajectory[i])] = 0
        text_list.append('        ' * position_count[str(trajectory[i])] + f'{i}')
    text_list[0] = 'Start 0'
    text_list[-1] = f'End {trajectory.shape[0] - 1}'

    fig.add_trace(
        go.Scattergl(
            x=trajectory[:, 0],
            y=trajectory[:, 1],
            mode=mode,
            marker=dict(size=size_list, color=color_list),
            line=dict(shape='linear', color='lightgrey', width=3, dash='dash'),
            text=text_list,
            textposition="top center",
            name='trajectory',
        ))
    
    if trajectory2 is not None:
        size_list2 = [6] * trajectory2.shape[0]
        size_list2[0] = 10
        size_list2[-1] = 10
        
        color_list2 = ['rgba(4, 4, 174, 0.5)'] * trajectory2[0].shape[0]
        color_list2[0] = 'rgba(235, 5, 5, 1)'
        color_list2[-1] = 'rgba(5, 235, 5, 1)'
        fig.add_trace(
            go.Scattergl(
                x=trajectory2[:, 0],
                y=trajectory2[:, 1],
                mode=mode,
                marker=dict(size=size_list2, color=color_list2),
                line=dict(shape='linear', color='red', width=3, dash='dash'),
                #text=text_list,
                textposition="top center",
                name='trajectory',
            ))

    # add floor plan
    floor_plan = Image.open(floor_plan_filename)
    fig.update_layout(images=[
        go.layout.Image(
            source=floor_plan,
            xref="x",
            yref="y",
            x=0,
            y=height_meter,
            sizex=width_meter,
            sizey=height_meter,
            sizing="contain",
            opacity=1,
            layer="below",
        )
    ])

    # configure
    fig.update_xaxes(autorange=False, range=[0, width_meter])
    fig.update_yaxes(autorange=False, range=[0, height_meter], scaleanchor="x", scaleratio=1)
    fig.update_layout(
        title=go.layout.Title(
            text=title or "No title.",
            xref="paper",
            x=0,
        ),
        autosize=True,
        width=800,
        height=  800 * height_meter / width_meter,
        template="plotly_white",
    )

    if show:
        fig.show()

    return fig

def visualize_train_trajectory(txt_path):
    """
    Edited from 
    https://www.kaggle.com/hrshtt/intro-to-indoor-location-navigation/
    who Edited from 
    https://www.kaggle.com/ihelon/indoor-location-exploratory-data-analysis
    """
    if not isinstance(txt_path, Path):
        path = Path(txt_path)
    
    _id, floor, filename = txt_path.parts[-3:]

    
    train_floor_data = read_data_file(txt_path)
    with open(f"../input/indoor-location-navigation/metadata/{_id}/{floor}/floor_info.json") as f:
        train_floor_info = json.load(f)

    return visualize_trajectory(
        train_floor_data.waypoint[:, 1:3], 
        f"../input/indoor-location-navigation/metadata/{_id}/{floor}/floor_image.png",
        train_floor_info["map_info"]["width"], 
        train_floor_info["map_info"]["height"],
        f"Visualization of {_id}/{floor}/{filename}"
    )

def visualize_train_step_trajectory(txt_path, step_positions):
    """
    Edited from 
    https://www.kaggle.com/hrshtt/intro-to-indoor-location-navigation/
    who Edited from 
    https://www.kaggle.com/ihelon/indoor-location-exploratory-data-analysis
    """
#     _id, floor = path.split("/")[:2]
    if not isinstance(txt_path, Path):
        path = Path(txt_path)
    
    _id, floor, filename = txt_path.parts[-3:]

    
    train_floor_data = read_data_file(txt_path)
    with open(f"../input/indoor-location-navigation/metadata/{_id}/{floor}/floor_info.json") as f:
        train_floor_info = json.load(f)
        
    return visualize_trajectory(
        train_floor_data.waypoint[:, 1:3], 
        f"../input/indoor-location-navigation/metadata/{_id}/{floor}/floor_image.png",
        train_floor_info["map_info"]["width"], 
        train_floor_info["map_info"]["height"],
        f"Visualization of {_id}/{floor}/{filename}", 
        trajectory2 = step_positions[:, 1:] #, step_positions[:, 2]
    )


In [ ]:
sample_txt_file_path = Path('../input/indoor-location-navigation/train/5a0546857ecc773753327266/B1/5e15730aa280850006f3d005.txt')
sample_txt_file_path

Predicts the position given the IMU and the starting point

In [ ]:
def predict_position(trajectory_file): 
    # Copied and modified from https://github.com/location-competition/indoor-location-competition-20/blob/master/compute_f.py
    sample_data = read_data_file(str(trajectory_file))
    
    # GT position 
    waypoint_df = pd.DataFrame(sample_data.waypoint)
    waypoint_df.columns = ['timestamp', 'waypoint_x','waypoint_y']
    
    # Compute relative position
    step_timestamps, step_indexs, step_acce_max_mins = compute_steps(sample_data.acce)
    headings = compute_headings(sample_data.ahrs)
    stride_lengths = compute_stride_length(step_acce_max_mins)
    step_headings = compute_step_heading(step_timestamps, headings)
    rel_positions = compute_rel_positions(stride_lengths, step_headings)
    
    rel_positions[:, 1].cumsum(), rel_positions[:, 2].cumsum()
    
    predicted_positions = np.zeros(rel_positions.shape)
    predicted_positions[:, 0] = rel_positions[:, 0]
    predicted_positions[:, 1] = rel_positions[:, 1].cumsum() + waypoint_df.loc[0].waypoint_x
    predicted_positions[:, 2] = rel_positions[:, 2].cumsum() + waypoint_df.loc[0].waypoint_y
    
    return predicted_positions

def calc_errors(trajectory_file, predicted_positions=None, only_mean_error = True): 
    if predicted_positions is None: 
        predicted_positions = predict_position(trajectory_file)
       
    # GT position 
    sample_data = read_data_file(str(trajectory_file))
    
    errors = [] 
    time_diff = []
    start_time = sample_data.waypoint[0,0]
    for predicted_position in predicted_positions: 
        timediff = 10e15
        best_gt = -1
        for idx, gt in enumerate(sample_data.waypoint): 
            if abs(predicted_position[0] - gt[0]) < timediff: 
                timediff = abs(predicted_position[0] - gt[0])
                best_gt = gt
        errors.append(np.linalg.norm(predicted_position[1:] - best_gt[1:]))
        time_diff.append(predicted_position[0] - start_time)

    if only_mean_error: 
        return np.mean(errors)
    return errors, time_diff, np.mean(errors)

In [ ]:
predicted_positions = predict_position(sample_txt_file_path)

In [ ]:
calc_errors(sample_txt_file_path, predicted_positions)

In [ ]:
visualize_train_step_trajectory(Path(sample_txt_file_path), predicted_positions) 

Some samples 

In [ ]:
import random
for filename in random.sample(os.listdir('../input/indoor-location-navigation/train/5a0546857ecc773753327266/B1/'), 5): 
    sample_txt_file_path = os.path.join('../input/indoor-location-navigation/train/5a0546857ecc773753327266/B1/', filename)
    predicted_positions = predict_position(sample_txt_file_path)
    print(calc_errors(sample_txt_file_path, predicted_positions))
    visualize_train_step_trajectory(Path(sample_txt_file_path), predicted_positions).show()
    plt.show()

In [ ]:
mean_errors = []
basedir = '../input/indoor-location-navigation/train/5a0546857ecc773753327266/B1/'
for filename in os.listdir(basedir):
    sample_txt_file_path = os.path.join(basedir, filename)
    predicted_positions = predict_position(sample_txt_file_path)
    mean_errors.append(calc_errors(sample_txt_file_path, predicted_positions))
np.mean(mean_errors), np.min(mean_errors), np.max(mean_errors), np.std(mean_errors)

Calculate the error per floor and building 

In [ ]:
import collections
def calc_errors_per_building(building_path): 
    floor_mean_errors = collections.OrderedDict()
    for floor in os.listdir(building_path): 
        floor_mean_errors[floor] = []
        basedir = os.path.join(building_path, floor)
        for filename in os.listdir(basedir):
            sample_txt_file_path = os.path.join(basedir, filename)
            predicted_positions = predict_position(sample_txt_file_path)
            floor_mean_errors[floor] .append(calc_errors(sample_txt_file_path, predicted_positions))
    return floor_mean_errors

In [ ]:
floor_mean_errors = calc_errors_per_building('../input/indoor-location-navigation/train/5a0546857ecc773753327266/')

In [ ]:
labels, data = floor_mean_errors.keys(), floor_mean_errors.values()

plt.boxplot(data)
plt.xticks(range(1, len(labels) + 1), labels)
plt.show()

Calculate the errors for 10 sample buildings

In [ ]:
for building in random.sample(os.listdir('../input/indoor-location-navigation/train'), 10): 
    floor_mean_errors = calc_errors_per_building(os.path.join('../input/indoor-location-navigation/train', building))
    labels, data = floor_mean_errors.keys(), floor_mean_errors.values()
    plt.boxplot(data)
    plt.xticks(range(1, len(labels) + 1), labels)
    plt.show()